In [82]:
import os
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [83]:
df = pd.read_csv("../data/heart.csv")

In [84]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [85]:
df.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.366337,0.683168,0.966997,131.623762,246.264026,0.148515,0.528053,149.646865,0.326733,1.039604,1.399340,0.729373,2.313531,0.544554
std,9.082101,0.466011,1.032052,17.538143,51.830751,0.356198,0.525860,22.905161,0.469794,1.161075,0.616226,1.022606,0.612277,0.498835
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.500000,0.000000,0.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,55.000000,1.000000,1.000000,130.000000,240.000000,0.000000,1.000000,153.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,274.500000,0.000000,1.000000,166.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


In [86]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
 13  target    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


Here is a bit more fleshed out description of each variable:

1. **age**: The person's age in years
2. **sex**: The person's sex (1 = male, 0 = female)
3. **cp**: The chest pain experienced (Value 1: typical angina, Value 2: atypical angina, Value 3: non-anginal pain, Value 4: asymptomatic)
4. **trestbps**: The person's resting blood pressure (mm Hg on admission to the hospital)
5. **chol**: The person's cholesterol measurement in mg/dl
6. **bs**: The person's fasting blood sugar (> 120 mg/dl, 1 = true; 0 = false)
7. **restecg**: Resting electrocardiographic measurement (0 = normal, 1 = having ST-T wave abnormality, 2 = showing probable or definite left ventricular hypertrophy by Estes' criteria)
8. **thalach**: The person's maximum heart rate achieved
9. **exang**: Exercise induced angina (1 = yes; 0 = no)
10. **oldpeak**: ST depression induced by exercise relative to rest ('ST' relates to positions on the ECG plot. See more here)
11. **slope**: the slope of the peak exercise ST segment (Value 1: upsloping, Value 2: flat, Value 3: downsloping)
12. **ca**: The number of major vessels (0-3)
13. **thal**: A blood disorder called thalassemia (3 = normal; 6 = fixed defect; 7 = reversable defect)
14. **target**: Heart disease (0 = no, 1 = yes)

## Feature Engineering and Data Cleaning

Let's try to clean the column names of the data. If we name our columns properly, we will avoid confusion in the future during data cleaning or engineering new features.

In [87]:
df.columns = ['age', 'sex', 'chest_pain_type', 'resting_blood_pressure', 'cholesterol', 'fasting_blood_sugar', 'rest_ecg', 'max_heart_rate_achieved',
       'exercise_induced_angina', 'st_depression', 'st_slope', 'num_major_vessels', 'thalassemia', 'target']

We see that a lot of the features that are supposed to be categorical are in fact encoded as integers. Let's try to clean these up. 

By doing this, we can set up our data for visualizing as well as providing as good foundation for data preprocessing for training models.

In [212]:
def clean_data(df):
    df['sex'][df['sex'] == 0] = 'female'
    df['sex'][df['sex'] == 1] = 'male'

    df['chest_pain_type'][df['chest_pain_type'] == 0] = 'typical angina'
    df['chest_pain_type'][df['chest_pain_type'] == 1] = 'atypical angina'
    df['chest_pain_type'][df['chest_pain_type'] == 2] = 'non-anginal pain'
    df['chest_pain_type'][df['chest_pain_type'] == 3] = 'asymptomatic'

    df['fasting_blood_sugar'][df['fasting_blood_sugar'] == 0] = 'lower than 120mg/ml'
    df['fasting_blood_sugar'][df['fasting_blood_sugar'] == 1] = 'greater than 120mg/ml'

    df['rest_ecg'][df['rest_ecg'] == 0] = 'normal'
    df['rest_ecg'][df['rest_ecg'] == 1] = 'ST-T wave abnormality'
    df['rest_ecg'][df['rest_ecg'] == 2] = 'left ventricular hypertrophy'

    df['exercise_induced_angina'][df['exercise_induced_angina'] == 0] = 'no'
    df['exercise_induced_angina'][df['exercise_induced_angina'] == 1] = 'yes'

    df['st_slope'][df['st_slope'] == 0] = 'upsloping'
    df['st_slope'][df['st_slope'] == 1] = 'flat'
    df['st_slope'][df['st_slope'] == 2] = 'downsloping'

    df['thalassemia'][df['thalassemia'] == 0] = 'normal'
    df['thalassemia'][df['thalassemia'] == 1] = 'fixed defect'
    df['thalassemia'][df['thalassemia'] == 2] = 'reversable defect'
    df['thalassemia'][df['thalassemia'] == 3] = 'reversable defect'

    return df

In [89]:
def change_dtypes(df):
    df['sex'] = df['sex'].astype('object')
    df['chest_pain_type'] = df['chest_pain_type'].astype('object')
    df['fasting_blood_sugar'] = df['fasting_blood_sugar'].astype('object')
    df['rest_ecg'] = df['rest_ecg'].astype('object')
    df['exercise_induced_angina'] = df['exercise_induced_angina'].astype('object')
    df['st_slope'] = df['st_slope'].astype('object')
    df['thalassemia'] = df['thalassemia'].astype('object')
    
    return df


In [90]:
df = clean_data(df)
df = change_dtypes(df)

## Correlations between features

Let's try to see how our features are correlated with each other as well as the target variable.

In [101]:
fig = go.Figure(data=go.Heatmap(z=df.corr(), 
                                x=df.corr().columns, 
                                y=df.corr().columns, 
                                colorscale=px.colors.sequential.Blugrn, 
                                text=df.corr().values, 
                                
                                ))
fig.update_layout(height=600, 
                  width=600, 
                  title="Correlation Matrix Heatmap")
fig.show()

There seems to be weak to moderate correlations between the features. Therefore, we can retain all of the features for the purpose of traning.

In [92]:
#df = pd.get_dummies(df, drop_first=False)

## Visualizing our features

It's time to visualize our features! Let's write simple functions to reduce our effort.

In [209]:
def plot_single_feature(df, feature):
    fig = None
    if feature == "target" or feature == "num_major_vessels":
        fig = px.bar(y=df[feature].value_counts(), 
                     x=df[feature].value_counts().index, 
                     color=df[feature].value_counts().index.astype(str), 
                     text=df[feature].value_counts())
        fig.update_layout(showlegend=False, 
                          title="Distribution of {}".format(feature))
        
        fig.show()
        return

    if df[feature].dtype == 'int64' or df[feature].dtype == 'float64':
        fig = px.histogram(x=df[feature].values, nbins=0)

    elif df[feature].dtype == 'object':
        fig = px.bar(y=df[feature].value_counts(), 
                     x=df[feature].value_counts().index, 
                     color=df[feature].value_counts().index, 
                     text=df[feature].value_counts())
    fig.update_layout(showlegend=False, 
                      title="Distribution of {}".format(feature))
    fig.show()
    return 

In [211]:
for feature in df.columns:
    plot_single_feature(df, feature)